### PROJECT - Credit Card Fraud Detection

#### **6. Optuna Optimization for the Best Pipeline**

Our goal is to create an Optuna study that finds the best combination of:
1.  **Imbalance Handling Strategy**: SMOTE vs. Random Undersampling.
2.  **Classifier**: A fast linear model (`LogisticRegression`) vs. a powerful tree ensemble (`LightGBM`).
3.  **Hyperparameters** for the chosen classifier.

We will optimize for the **F1-score**, as it provides a good balance between precision and recall, which is a common starting point for imbalanced problems.

**Implementation:**

In [1]:
# imports

import optuna
import lightgbm as lgb
from sklearn.model_selection import (
    StratifiedKFold,
    cross_val_score
)
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression

In [2]:
import pandas as pd
df = pd.read_csv('creditcard.csv')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
scaler = StandardScaler()
df['scaled_amount'] = scaler.fit_transform(df['Amount'].values.reshape(-1, 1))
df['scaled_time'] = scaler.fit_transform(df['Time'].values.reshape(-1, 1))

# Drop the original columns
df.drop(['Time', 'Amount'], axis=1, inplace=True)

scaled_amount = df.pop('scaled_amount')
scaled_time = df.pop('scaled_time')
df.insert(0, 'scaled_amount', scaled_amount)
df.insert(1, 'scaled_time', scaled_time)

X = df.drop('Class', axis=1)
y = df['Class']

#  Use stratify=y, it's absolutely critical here.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

**Define the optuna objective function**

In [7]:
# This might be the most complex objective function yet, but it follows the same logic
def objective(trial):
    # 1. Suggest the imbalance handling strategy
    resampling_strategy = trial.suggest_categorical('resampling', ['smote','under', 'none'])
    
    # 2. Suggest the classifier
    classifier_name = trial.suggest_categorical('classifier', ['LogisticRegression', 'LightGBM'])

    # 3. Define the pipeline steps based on choices
    if resampling_strategy == 'smote':
        resampler = SMOTE(random_state=42)
    elif resampling_strategy == 'under':
        resampler = RandomUnderSampler(random_state=42)
    else:
        # Passthrough if no resampling is chosen
        resampler = 'passthrough'
    # Suggest Hyperparameters for the chosen classifier
    if classifier_name == 'LogisticRegression':
        C = trial.suggest_float('C', 1e-4, 1e2, log=True)
        # Add class_weight as an option, especially when 'resampling' is 'none'
        use_class_weight = trial.suggest_categorical('lr_class_weight', [True, False])
        
        class_weight = 'balanced' if use_class_weight else None

        classifier = LogisticRegression(C=C, class_weight=class_weight,
                                        random_state=42,
                                        solver='liblinear')

    else: # LIGHT GBM
        lgbm_params = {
            'objective': 'binary',
            'metric': 'binary_logloss',
            'n_estimators': trial.suggest_int('n_estimators', 200, 1000),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.03),
            'num_leaves': trial.suggest_int('num_leaves', 10, 100),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-4, 1e2, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
            'is_unbalance': True, # LightGBM has its own imbalance handling
            'n_jobs':-1,
            'random_state': 42
        }
        classifier = lgb.LGBMClassifier(**lgbm_params)
    
    # Create the full pipeline
    pipeline = ImbPipeline(steps=[
        ('resampler', resampler),
        ('classifier', classifier)
    ])
    # Evaluate using Cross validation

    cv = StratifiedKFold(n_splits=3, # Use 3 splits for speed
                         shuffle=True,
                         random_state=42) 
    score = cross_val_score(pipeline, X_train, y_train, scoring='f1', cv=cv, n_jobs=-1)
    
    return score.mean()


**Run the Optuna study**

In [8]:
# WE are maximizing F1
study = optuna.create_study(direction='maximize',
                             study_name='Fraud_detection_optimization')
# Let's run for 150 trials
study.optimize(objective, n_trials=150)
print(f"Best F1-score: {study.best_value:.4f}")
print("Best hyperparameters:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

[I 2025-11-15 19:27:22,182] A new study created in memory with name: Fraud_detection_optimization
[I 2025-11-15 19:27:31,711] Trial 0 finished with value: 0.11341150630779452 and parameters: {'resampling': 'none', 'classifier': 'LogisticRegression', 'C': 51.702938342843346, 'lr_class_weight': True}. Best is trial 0 with value: 0.11341150630779452.
[I 2025-11-15 19:27:37,926] Trial 1 finished with value: 0.09376313019820992 and parameters: {'resampling': 'under', 'classifier': 'LightGBM', 'n_estimators': 376, 'learning_rate': 0.018525199266155517, 'num_leaves': 94, 'max_depth': 9, 'min_child_samples': 34, 'subsample': 0.9966345736571192, 'colsample_bytree': 0.5750757766822803, 'reg_alpha': 0.0023104447002549166, 'reg_lambda': 0.08695316058063476}. Best is trial 0 with value: 0.11341150630779452.
[I 2025-11-15 19:27:47,603] Trial 2 finished with value: 0.10669999683904147 and parameters: {'resampling': 'smote', 'classifier': 'LogisticRegression', 'C': 0.09413480529543895, 'lr_class_weigh

Best F1-score: 0.8702
Best hyperparameters:
  resampling: none
  classifier: LightGBM
  n_estimators: 984
  learning_rate: 0.012884579931656037
  num_leaves: 68
  max_depth: 9
  min_child_samples: 86
  subsample: 0.8515735476743733
  colsample_bytree: 0.9614950229666329
  reg_alpha: 0.00021881931021067107
  reg_lambda: 0.002003625697370366


#### **7. Final Model Training and Threshold Tuning**

Now that Optuna has found the best pipeline configuration, we train it on the full training set and then perform the final, crucial step: tuning the decision threshold on the test set to meet our business goal (e.g., achieve 95% recall).

**Build and Train the Final pipeline with the best parameters**

In [14]:
best_params = study.best_params
final_resampling_Strategy = best_params.pop('resampling')
print(final_resampling_Strategy)
final_classifier_name = best_params.pop('classifier')
print(final_classifier_name)
if final_resampling_Strategy == 'smote':
    final_resampler = SMOTE(random_state=42)
elif final_resampling_Strategy == 'under':
    final_resampler = RandomUnderSampler(random_state=42)
else:
    final_resampler = 'passthrough'

if final_classifier_name == 'LogisticRegression':
    use_class_weight = best_params.pop('lr_class_weight', None)
    print(use_class_weight)
    if use_class_weight:
        best_params['class_weight'] = 'balanced'
    final_classifier = LogisticRegression(random_state=42,
                                          solver='liblinear', **best_params)
else:
    # Remove any irrelevant params from the dict before passing to LGBM
    lgbm_keys = ['objective', 'metric', 'n_estimators', 'learning_rate', 'num_leaves', 'max_depth', 'min_child_samples', 'subsample', 'colsample_bytree', 'reg_alpha', 'reg_lambda', 'is_unbalance']
    
    lgbm_params = {k:v for k,v in best_params.items() if k in lgbm_keys}
    final_classifier = lgb.LGBMClassifier(objective='binary',
                                          metric='binary_logloss',
                                          is_unbalance = True,
                                          n_jobs=-1,
                                          random_state=42,
                                          **lgbm_params)
    final_pipeline = ImbPipeline(steps=[
    ('resampler', final_resampler),
    ('classifier', final_classifier)
])

final_pipeline.fit(X_train, y_train)

    

none
LightGBM
[LightGBM] [Info] Number of positive: 394, number of negative: 227451
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.039807 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7650
[LightGBM] [Info] Number of data points in the train set: 227845, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001729 -> initscore=-6.358339
[LightGBM] [Info] Start training from score -6.358339
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wit

,steps,"[('resampler', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,boosting_type,'gbdt'
,num_leaves,68
,max_depth,9
,learning_rate,0.012884579931656037
,n_estimators,984
,subsample_for_bin,200000
,objective,'binary'


**Threshold Tuning**


In [1]:
from sklearn.metrics import precision_recall_curve, recall_score, precision_score, f1_score

# Get the predicted probabilities for the positive class (Fraud)
y_proba_final = final_pipeline.predict_proba(X_test)[:, 1]

# Calculate precision and recall for all possible thresholds
precisions, recalls, thresholds = precision_recall_curve(y_test, y_proba_final)

# Find the threshold that gives us at least 95% recall
target_recall = 0.95


NameError: name 'final_pipeline' is not defined